In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [12]:
import pandas as pd
import numpy as np
import random

# Function to generate random fuel distribution satisfying constraints
# Function to generate random fuel distribution satisfying constraints
def generate_fuel_distribution(total_fuel):
    # Initialize the fuel distribution list
    fuel_distribution = [0] * 6
    
    # Generate fuel amounts that are multiples of either 4 or 3
    for i in range(6):
        # Generate a random fuel amount
        fuel = np.random.choice(range(total_fuel + 1))
        
        # Ensure the fuel amount is a multiple of either 4 or 3
        while fuel % 4 != 0 and fuel % 3 != 0:
            fuel = np.random.choice(range(total_fuel + 1))
        
        fuel_distribution[i] = fuel
        total_fuel -= fuel
    
    return fuel_distribution

# Generate sample dataset
def generate_sample_dataset(num_locations, customers_per_location):
    data = []
    for location in range(1, num_locations + 1):
        for customer in range(1, customers_per_location + 1):
            num_orders = random.randint(1, 8)  # Randomly select number of orders
            for _ in range(num_orders):
                #date =  "04-04-2024"


                current_date = pd.Timestamp.now().date()

                # Define the start date as two years ago from the current date
                start_date = current_date - pd.DateOffset(years=2)

                # Define the end date as yesterday to ensure that the last two years are included
                end_date = current_date - pd.DateOffset(days=1)

                # Generate a random date between start_date and end_date
                date = pd.Timestamp(start_date + pd.Timedelta(days=np.random.randint((end_date - start_date).days)))

                time = random.choice(['08:15', '09:30', '11:45', '13:00', '16:20', '17:40'])
                customer_id = str(location).zfill(2) + str(customer).zfill(3)  # Unique customer IDs based on location
                total_fuel = random.choice([12, 24])
                fuel_distribution = generate_fuel_distribution(total_fuel)
                data.append([date, time, customer_id, location, total_fuel] + list(fuel_distribution))
    columns = ['Date', 'Time', 'CustomerID', 'Location', 'Total Fuel', 'Fuel_1', 'Fuel_2', 'Fuel_3', 'Fuel_4', 'Fuel_5', 'Fuel_6']
    return pd.DataFrame(data, columns=columns)

# Generate sample dataset with 30 locations, 10 customers per location, and variable number of orders
sample_data = generate_sample_dataset(15, 10)

# Shuffle the dataset
sample_data = sample_data.sample(frac=1).reset_index(drop=True)

# Save sample dataset to CSV file
sample_data.to_csv('sample_dataset.csv', index=False)

# Display sample dataset
print(sample_data)


          Date   Time CustomerID  Location  Total Fuel  Fuel_1  Fuel_2  \
0   2024-02-20  17:40      14007        14          24      24       0   
1   2023-06-18  08:15      01006         1          24      24       0   
2   2023-07-19  11:45      03009         3          12       3       3   
3   2023-05-15  08:15      11002        11          12       0       3   
4   2022-06-10  17:40      04003         4          12       6       0   
..         ...    ...        ...       ...         ...     ...     ...   
699 2023-09-09  11:45      07001         7          12       6       4   
700 2022-06-06  08:15      15007        15          12       3       6   
701 2023-01-30  11:45      05002         5          12      12       0   
702 2023-08-14  16:20      14009        14          24      16       3   
703 2022-11-05  13:00      04002         4          12       6       3   

     Fuel_3  Fuel_4  Fuel_5  Fuel_6  
0         0       0       0       0  
1         0       0       0       0

In [ ]:
len(np.unique(sample_data["CustomerID"]))

In [ ]:
sample_data.to_csv('sample_dataset.csv', index=False)

In [13]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense

# Load dataset
data = pd.read_csv('sample_dataset.csv')

# Convert date and time to datetime format
data['Date'] = pd.to_datetime(data['Date'] + ' ' + data['Time'])

# Extract features
data['DayOfWeek'] = data['Date'].dt.dayofweek
data['Month'] = data['Date'].dt.month
data['Hour'] = data['Date'].dt.hour
data['Minute'] = data['Date'].dt.minute

# Normalize numerical features
scaler = MinMaxScaler()
data[['DayOfWeek', 'Month', 'Hour', 'Minute', 'Fuel_1', 'Fuel_2', 'Fuel_3', 'Fuel_4', 'Fuel_5', 'Fuel_6']] = scaler.fit_transform(data[['DayOfWeek', 'Month', 'Hour', 'Minute', 'Fuel_1', 'Fuel_2', 'Fuel_3', 'Fuel_4', 'Fuel_5', 'Fuel_6']])

# Define features and target
X = data[['DayOfWeek', 'Month', 'Hour', 'Minute']].values
y_total = data['Total Fuel'].values
y_distribution = data[['Fuel_1', 'Fuel_2', 'Fuel_3', 'Fuel_4', 'Fuel_5', 'Fuel_6']].values

# Split data into train and test sets
X_train, X_test, y_train_total, y_test_total, y_train_distribution, y_test_distribution = train_test_split(X, y_total, y_distribution, test_size=0.2, shuffle=False)

# Reshape input data for LSTM
X_train = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
X_test = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))

# Define LSTM model for total fuel demand prediction
model_total = Sequential()
model_total.add(LSTM(units=50, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2])))
model_total.add(Dense(units=1))
model_total.compile(optimizer='adam', loss='mse')

# Train the model for total fuel demand prediction
history_total = model_total.fit(X_train, y_train_total, epochs=50, batch_size=32, validation_data=(X_test, y_test_total), verbose=1)

# Define LSTM model for fuel distribution prediction
model_distribution = Sequential()
model_distribution.add(LSTM(units=50, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2])))
model_distribution.add(Dense(units=6))  # Output 6 values, one for each fuel type
model_distribution.compile(optimizer='adam', loss='mse')

# Train the model for fuel distribution prediction
history_distribution = model_distribution.fit(X_train, y_train_distribution, epochs=50, batch_size=32, validation_data=(X_test, y_test_distribution), verbose=1)

# Evaluate the models
mse_total = model_total.evaluate(X_test, y_test_total)
mse_distribution = model_distribution.evaluate(X_test, y_test_distribution)
print("Mean Squared Error (Total Fuel):", mse_total)
print("Mean Squared Error (Fuel Distribution):", mse_distribution)

# Make predictions
predictions_total = model_total.predict(X_test)
predictions_distribution = model_distribution.predict(X_test)

# Visualize predictions vs actual values
# (You can use matplotlib or any other visualization library for this)


2024-04-04 07:57:19.790026: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-04 07:57:19.790165: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-04 07:57:19.997541: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Epoch 1/50


/opt/conda/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:205: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


18/18 ━━━━━━━━━━━━━━━━━━━━ 2s 20ms/step - loss: 355.7877 - val_loss: 349.8616
Epoch 2/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 363.1908 - val_loss: 345.7569
Epoch 3/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 350.6413 - val_loss: 339.5614
Epoch 4/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 349.1295 - val_loss: 330.4029
Epoch 5/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 329.7110 - val_loss: 318.0803
Epoch 6/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 321.2045 - val_loss: 302.1480
Epoch 7/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 291.8257 - val_loss: 282.7787
Epoch 8/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 274.6163 - val_loss: 259.6769
Epoch 9/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 257.2163 - val_loss: 233.7383
Epoch 10/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 224.9709 - val_loss: 206.1861
Epoch 11/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 194.9309 - val_loss: 178.2029
Epoch 12/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 0

In [14]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense

# Load dataset
data = pd.read_csv('sample_dataset.csv')

# Convert date and time to datetime format
data['Date'] = pd.to_datetime(data['Date'] + ' ' + data['Time'])

# Extract features
data['DayOfWeek'] = data['Date'].dt.dayofweek
data['Month'] = data['Date'].dt.month
data['Hour'] = data['Date'].dt.hour
data['Minute'] = data['Date'].dt.minute

# Normalize numerical features
scaler = MinMaxScaler()
data[['DayOfWeek', 'Month', 'Hour', 'Minute', 'Fuel_1', 'Fuel_2', 'Fuel_3', 'Fuel_4', 'Fuel_5', 'Fuel_6']] = scaler.fit_transform(data[['DayOfWeek', 'Month', 'Hour', 'Minute', 'Fuel_1', 'Fuel_2', 'Fuel_3', 'Fuel_4', 'Fuel_5', 'Fuel_6']])

# Define features and target
X = data[['DayOfWeek', 'Month', 'Hour', 'Minute']].values
y_total = data['Total Fuel'].values

# Convert y_total to binary labels
y_binary = (y_total == 24).astype(int)

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y_binary, test_size=0.2, shuffle=False)

# Reshape input data for LSTM
X_train = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
X_test = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))

# Define LSTM model for total fuel demand classification
model = Sequential()
model.add(LSTM(units=50, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dense(units=1, activation='sigmoid'))  # Output probability for binary classification
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])  # Use binary cross-entropy loss and accuracy metric

# Train the model
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test), verbose=1)

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print("Loss:", loss)
print("Accuracy:", accuracy)

# Make predictions
predictions_proba = model.predict(X_test)
predictions_binary = (predictions_proba > 0.5).astype(int)

# Visualize predictions vs actual values
# (You can use classification metrics and confusion matrix for evaluation)


Epoch 1/50


/opt/conda/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:205: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


18/18 ━━━━━━━━━━━━━━━━━━━━ 2s 22ms/step - accuracy: 0.4916 - loss: 0.6945 - val_accuracy: 0.4681 - val_loss: 0.6947
Epoch 2/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5149 - loss: 0.6920 - val_accuracy: 0.4894 - val_loss: 0.6944
Epoch 3/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.4884 - loss: 0.6930 - val_accuracy: 0.4894 - val_loss: 0.6941
Epoch 4/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.5141 - loss: 0.6928 - val_accuracy: 0.4397 - val_loss: 0.6940
Epoch 5/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.5374 - loss: 0.6919 - val_accuracy: 0.4539 - val_loss: 0.6941
Epoch 6/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5083 - loss: 0.6921 - val_accuracy: 0.4610 - val_loss: 0.6941
Epoch 7/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.5018 - loss: 0.6922 - val_accuracy: 0.4681 - val_loss: 0.6942
Epoch 8/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5088 - loss: 0.6928 - val_accuracy: 0.4681 - val_loss: 0.6943
Ep

In [15]:
predictions_proba

array([[0.53768486],
       [0.51402575],
       [0.5158247 ],
       [0.46139425],
       [0.47766173],
       [0.51296896],
       [0.47433767],
       [0.48462474],
       [0.46367228],
       [0.47343493],
       [0.5605266 ],
       [0.5420498 ],
       [0.4681668 ],
       [0.44781366],
       [0.43916088],
       [0.48501638],
       [0.4742319 ],
       [0.5555755 ],
       [0.51244867],
       [0.5595331 ],
       [0.5555755 ],
       [0.4913653 ],
       [0.4841042 ],
       [0.5300475 ],
       [0.511459  ],
       [0.5026263 ],
       [0.47513908],
       [0.4827322 ],
       [0.49509165],
       [0.4712961 ],
       [0.48931402],
       [0.574765  ],
       [0.46699643],
       [0.48565888],
       [0.5071384 ],
       [0.5269627 ],
       [0.5160867 ],
       [0.4993001 ],
       [0.49018088],
       [0.52984416],
       [0.5595331 ],
       [0.4848653 ],
       [0.4827322 ],
       [0.57417876],
       [0.47993243],
       [0.48286134],
       [0.5068337 ],
       [0.559

In [19]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from keras.optimizers import Adam

# Load dataset
data = pd.read_csv('sample_dataset.csv')

# Convert date to datetime format
data['Date'] = pd.to_datetime(data['Date'])

# Extract hour and minute from 'Time' column and combine with date to create a datetime object
data['Time'] = pd.to_datetime(data['Time'], format='%H:%M')
data['Datetime'] = pd.to_datetime(data['Date'].astype(str) + ' ' + data['Time'].dt.strftime('%H:%M:%S'))

# Extract features from datetime
data['DayOfWeek'] = data['Datetime'].dt.dayofweek
data['Month'] = data['Datetime'].dt.month
data['Hour'] = data['Datetime'].dt.hour
data['Minute'] = data['Datetime'].dt.minute

# Normalize numerical features
scaler = MinMaxScaler()
data[['DayOfWeek', 'Month', 'Hour', 'Minute']] = scaler.fit_transform(data[['DayOfWeek', 'Month', 'Hour', 'Minute']])

# Define features and target
X = data[['DayOfWeek', 'Month', 'Hour', 'Minute', 'Location', 'CustomerID']].values
y = data[['Fuel_1', 'Fuel_2', 'Fuel_3', 'Fuel_4', 'Fuel_5', 'Fuel_6']].values

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

# Reshape input data for LSTM (samples, timesteps, features)
X_train = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
X_test = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))

# Define LSTM model architecture
model = Sequential()
model.add(LSTM(units=64, input_shape=(X_train.shape[1], X_train.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(units=32, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(units=16))
model.add(Dropout(0.2))
model.add(Dense(units=6, activation='softmax'))  # Output layer with softmax activation for multi-class classification

# Compile the model
optimizer = Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test), verbose=1)

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print("Loss:", loss)
print("Accuracy:", accuracy)

# Make predictions
predictions = model.predict(X_test)

# Visualize predictions vs actual values
# (You can use classification metrics and confusion matrix for evaluation)


Epoch 1/50


/opt/conda/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:205: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


18/18 ━━━━━━━━━━━━━━━━━━━━ 6s 42ms/step - accuracy: 0.1685 - loss: 30.6286 - val_accuracy: 0.6525 - val_loss: 29.9325
Epoch 2/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.5557 - loss: 29.9467 - val_accuracy: 0.6525 - val_loss: 28.5034
Epoch 3/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.5700 - loss: 27.8556 - val_accuracy: 0.6525 - val_loss: 25.8902
Epoch 4/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.5892 - loss: 25.9868 - val_accuracy: 0.6525 - val_loss: 23.4369
Epoch 5/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.5986 - loss: 24.5745 - val_accuracy: 0.6525 - val_loss: 22.5429
Epoch 6/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.6169 - loss: 24.6375 - val_accuracy: 0.6525 - val_loss: 22.4700
Epoch 7/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.5779 - loss: 23.4054 - val_accuracy: 0.6525 - val_loss: 22.5416
Epoch 8/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.6033 - loss: 23.1656 - val_accuracy: 0.6525 - val_

In [20]:
predictions

array([[7.3769045e-01, 1.5793031e-01, 4.0080748e-02, 5.8411822e-02,
        5.4131513e-03, 4.7360934e-04],
       [7.3769045e-01, 1.5793031e-01, 4.0080748e-02, 5.8411822e-02,
        5.4131513e-03, 4.7360934e-04],
       [7.3769045e-01, 1.5793031e-01, 4.0080748e-02, 5.8411822e-02,
        5.4131513e-03, 4.7360934e-04],
       [7.3769045e-01, 1.5793031e-01, 4.0080748e-02, 5.8411822e-02,
        5.4131513e-03, 4.7360934e-04],
       [7.3769045e-01, 1.5793031e-01, 4.0080748e-02, 5.8411822e-02,
        5.4131513e-03, 4.7360934e-04],
       [7.3769045e-01, 1.5793031e-01, 4.0080748e-02, 5.8411822e-02,
        5.4131513e-03, 4.7360934e-04],
       [7.3769045e-01, 1.5793031e-01, 4.0080748e-02, 5.8411822e-02,
        5.4131513e-03, 4.7360934e-04],
       [7.3769045e-01, 1.5793031e-01, 4.0080748e-02, 5.8411822e-02,
        5.4131513e-03, 4.7360934e-04],
       [7.3769045e-01, 1.5793031e-01, 4.0080748e-02, 5.8411822e-02,
        5.4131513e-03, 4.7360934e-04],
       [7.3769045e-01, 1.5793031e-01,

In [21]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from keras.optimizers import Adam

# Load dataset
data = pd.read_csv('sample_dataset.csv')

# Convert date to datetime format
data['Date'] = pd.to_datetime(data['Date'])

# Extract hour and minute from 'Time' column and combine with date to create a datetime object
data['Time'] = pd.to_datetime(data['Time'], format='%H:%M')
data['Datetime'] = pd.to_datetime(data['Date'].astype(str) + ' ' + data['Time'].dt.strftime('%H:%M:%S'))

# Extract features from datetime
data['DayOfWeek'] = data['Datetime'].dt.dayofweek
data['Month'] = data['Datetime'].dt.month
data['Hour'] = data['Datetime'].dt.hour
data['Minute'] = data['Datetime'].dt.minute

# Normalize numerical features
scaler = MinMaxScaler()
data[['DayOfWeek', 'Month', 'Hour', 'Minute']] = scaler.fit_transform(data[['DayOfWeek', 'Month', 'Hour', 'Minute']])

# Define features and target
X = data[['DayOfWeek', 'Month', 'Hour', 'Minute', 'Location', 'CustomerID']].values
y = data[['Fuel_1', 'Fuel_2', 'Fuel_3', 'Fuel_4', 'Fuel_5', 'Fuel_6']].values

# Ensure fuel amounts are multiples of 3 or 4
y = np.round(y / 3) * 3

# Calculate total fuel amounts
total_fuel = np.sum(y, axis=1)

# Define target total fuel values (either 12 or 24, whichever is closer)
target_total_fuel = np.where(total_fuel < 18, 12, 24)

# Adjust predictions to meet total fuel constraints
for i in range(len(y)):
    excess_fuel = np.abs(total_fuel[i] - target_total_fuel[i])
    while excess_fuel > 0:
        if total_fuel[i] < target_total_fuel[i]:
            # Increase one of the fuel types by 3 or 4 to meet the target
            fuel_idx = np.random.choice(np.where(y[i] % 3 == 0)[0])
            y[i][fuel_idx] += 3
            excess_fuel -= 3
        else:
            # Decrease one of the fuel types by 3 or 4 to meet the target
            fuel_idx = np.random.choice(np.where(y[i] % 3 == 0)[0])
            y[i][fuel_idx] -= 3
            excess_fuel -= 3

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

# Reshape input data for LSTM (samples, timesteps, features)
X_train = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
X_test = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))

# Define LSTM model architecture for classification
model = Sequential()
model.add(LSTM(units=64, input_shape=(X_train.shape[1], X_train.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(units=32, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(units=16))
model.add(Dropout(0.2))
model.add(Dense(units=6, activation='softmax'))  # Output layer with softmax activation for multi-class classification

# Compile the model
optimizer = Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test), verbose=1)

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print("Loss:", loss)
print("Accuracy:", accuracy)

# Make predictions
predictions = model.predict(X_test)

# Round the predictions to the nearest multiples of 3 or 4
predictions = np.round(predictions / 3) * 3

# Visualize predictions vs actual values
# (You can use classification metrics and confusion matrix for evaluation)


Epoch 1/50


/opt/conda/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:205: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


18/18 ━━━━━━━━━━━━━━━━━━━━ 6s 42ms/step - accuracy: 0.2011 - loss: 33.0750 - val_accuracy: 0.6950 - val_loss: 31.3477
Epoch 2/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.6004 - loss: 31.6517 - val_accuracy: 0.6950 - val_loss: 30.4037
Epoch 3/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.6403 - loss: 30.5124 - val_accuracy: 0.6950 - val_loss: 28.7353
Epoch 4/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.6575 - loss: 28.6865 - val_accuracy: 0.6950 - val_loss: 27.5331
Epoch 5/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.6367 - loss: 28.5518 - val_accuracy: 0.6950 - val_loss: 28.7785
Epoch 6/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.6532 - loss: 28.7193 - val_accuracy: 0.6950 - val_loss: 30.6188
Epoch 7/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.6403 - loss: 32.0493 - val_accuracy: 0.6950 - val_loss: 31.4183
Epoch 8/50
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.5925 - loss: 34.3017 - val_accuracy: 0.6950 - val_

In [22]:
predictions

array([[0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0.